# Project 1: Gauss
---
- *Name*: **Lê Võ Minh Phương**
- *Student ID*: **22120286**
- *Class*: **Toán ứng dụng thống kê 22_3**
---

**Requirements**  
- All works must be done and submitted in one .ipynb file
- Must include student's information and algorithms like in Practice 1.
- Must include descriptions and ideas for each function in the project. 

***Algorithm explanation*** Present the algorithms and ideas to implement two main functions `Gauss_elimination` and `back_substitution`.

### 1. Gaussian Elimination Algorithm:
   - **Input**: Receive the augmented matrix representing the system of linear equations.
   - **Swap Rows**: Swap rows to move rows with fewer leading zeros to the top.
   - **Check Echelon Form**: Verify if the matrix is already in row echelon form. If so, stop.
   - **Perform Gaussian Elimination**:
     - Iterate through each row of the augmented matrix:
       - Determine the number of non-zero entries in the current row (`nonZeros_cur_row`).
       - Find the number of non-zero entries in the previous row (`nonZeros_prev_row`).
       - If the number of non-zero entries in the current row matches the previous row:
         - Calculate the position to subtract (`pos_to_substract`) based on the count of leading zeros.
         - Compute the (`multiple`) as the ratio of elements in the current row and the corresponding element in the previous row.
         - Subtract the multiple times the previous row from the current row.
   - **Swap Rows (Post-Elimination)**: Swap rows again to maintain the desired order after Gaussian elimination.
   - **Output**: Return the modified augmented matrix.

### 2. Back Substitution Algorithm:
   - **Input**: Receive the row echelon matrix from Gaussian elimination.
   - **Preprocessing**:
     - Remove rows containing only zeros, as they are unnecessary and could cause division by zero errors.
   - **Initialize Variables**:
     - Determine the number of rows (`n`) and columns (`m`) in the augmented matrix.
     - Calculate the number of free variables (`num_free_variables`) in the system.
     - Initialize an empty list `result` to store the solution.
   - **Identify Free Variables**:
     - Generate rows in `result` representing free variables by setting coefficients of free variables to 1 and others to 0.
   - **Perform Back Substitution**:
     - Iterate through rows of the augmented matrix in reverse order (from `n-1` to `0`).
     - Initialize a vector `l` to represent the constants in the equations, with the last element being the constant term from the augmented matrix.
     - Perform row operations to eliminate variables and find the solution.
     - Perform division to isolate the variable being solved for.
   - **Print Solutions**:
     - Print the solutions with appropriate formatting, including the variable names and their corresponding coefficients.
   - **Output**: Print the solutions obtained from back substitution.


In [283]:
import numpy as np

## Task 1:
Input the system of equations, the same as an augmented matrix `[A|b]`

In [286]:
def inputSystemOfEquations():
    n = int(input("Enter the number of equations: "))
    sys_equations = []
    for i in range(n):
        equation = input(f"Enter the {i+1}th equation (e.g.: '3 6 9' for 3x + 6y = 9): ")
        sys_equations.append([float(x) for x in equation.split()])
    return sys_equations

## Task 2:
Write some utility functions for the Gauss_elimination

### 1: Function to count the number of zeros before the first non-zero entry of a row

In [290]:
def countAllZerosBeforeNonZero(sequence):
    count = 0
    for element in sequence:
        if element == 0:
            count += 1
        else:
            break
    return count
    

### 2. Function to check if a given matrix is an echelon matrix
It checks based on the rule that the `row[i+1]` has the position of the leading entry on the **right** of the leading entry of `row[i]` (means the number of zeros before a non-zero number of `row[i]` less than `row[i+1]`'s), excluding the case zero-row (which will be swapped with another function).

In [293]:
def checkNotAllZero(row):
    return all(entry == 0 for entry in row)

def isEchelonMatrix(matrix):
    num_rows = len(matrix)
    for row in range(num_rows - 1):
        if (not checkNotAllZero(matrix[row])) and (countAllZerosBeforeNonZero(matrix[row]) >= countAllZerosBeforeNonZero(matrix[row + 1])):
            return False
    return True

### 3. Function to swap rows with more zero entries to the bottom 

In [295]:
def swapRow(matrix):
    for i in range(0,len(matrix)-1):
        for j in range(i+1,len(matrix)):
            if(countAllZerosBeforeNonZero(matrix[i]) > countAllZerosBeforeNonZero(matrix[j])):
                matrix[i], matrix[j] = matrix[j], matrix[i]
    return matrix

## Task 3:
Write the Gauss_elimination to perform Gaussian elimination with the input is the entered augmented matrix `[A|b]` and return an row echelon matrix

In [298]:
def Gauss_elimination(augmented_matrix):
    # Step 1: Swap rows to move rows with fewer leading zeros to the top
    swapRow(augmented_matrix)

    # Step 2: Check if the matrix is already in row echelon form
    if isEchelonMatrix(augmented_matrix):
        return augmented_matrix
    # Step 3: Perform Gaussian elimination
    num_rows = len(augmented_matrix)
    for pivot_row_index in range(1, num_rows+1):
        if isEchelonMatrix(augmented_matrix):
            break
        for row_index in range(pivot_row_index, num_rows):
            nonZeros_cur_row = len(augmented_matrix[row_index]) - countAllZerosBeforeNonZero(augmented_matrix[row_index])
            nonZeros_prev_row = len(augmented_matrix[pivot_row_index - 1]) - countAllZerosBeforeNonZero(augmented_matrix[pivot_row_index - 1])
            # where the leading nonzero entry in each row is to the right of
            # the leading nonzero entry in the row above it.
            if nonZeros_prev_row == nonZeros_cur_row:
                pos_to_substract = countAllZerosBeforeNonZero(augmented_matrix[row_index])
                multiple = augmented_matrix[row_index][pos_to_substract] / augmented_matrix[pivot_row_index - 1][pos_to_substract]
                for col_index in range(0, len(augmented_matrix[0])):
                    augmented_matrix[row_index][col_index] -= multiple * augmented_matrix[pivot_row_index - 1][col_index]

        # Step 4: Swap rows again after Gaussian elimination
        swapRow(augmented_matrix)

    return augmented_matrix


## Task 4:
Write `back_substitution` to solve the linear system of equations, using the row echelon matrix from `Gauss_elimination`.\
*Note:* the `format_sequence` function is used to display the solutions in case of infinite solutions, with the free variables expressed as `t_1, t_2`,... belong to the set of real numbers `R`.

In [301]:
def format_sequence(seq):
    terms = []
    for i, coefficient in enumerate(seq):
        if i < len(seq) - 1:
            terms.append(f"{coefficient}*t_{i+1}")
    t = " + ".join(terms)
    t += f" + {seq[-1]}"
    return t
    
def back_substitution(M):
    #remove 0 rows, which are unnecessary for substitution and cause
    #errors with division by 0.
    M = [row for row in M if not all(value == 0 for value in row)]
   
    n = len(M)
    m = len(M[0])
    num_free_variables = m - 1 - n
    
    ## Initialize the result list to store the solution.
    result = []
    for _ in range(m-1):
        row = [0] * (num_free_variables+1) # Initialize each row with zeros
        result.append(row)
    
    # Create rows to represent free variables in the solution.
    for i in range(num_free_variables):
        row = [0] * (num_free_variables+1)
        row[num_free_variables-1-i] = 1 # Set the coefficient of each free variable to 1
        
        #Since the result array is initialized in reverse order (from the bottom up), m-2-i calculates the index in the result array
        #corresponding to the current row being processed.
        result[m-2-i] = row
        
    for i in range(n-1,-1,-1):# Iterate over rows in reverse order
      l = [0]*(num_free_variables+1)
      l[-1] = M[i][-1]

    # Convert the list to a numpy array for easier manipulation
      l1 = np.array(l)

    #Perform row operations to eliminate variables and find the solution.
      for j in range(len(result)):
        l1 = l1 - M[i][m-2-j]*np.array(result[m-2-j])
    #Perform division to isolate the variable being solved for
      l1 = l1/M[i][i]
      result[i] = list(l1)
    for i in range(len(result)):
      print(f"x_{i+1} = {format_sequence(result[i])}")

The `printNumberOfSol` is used to notify the users how many solutions the equation system has based on `Kronecker-Capelli Theorem`.

In [304]:
def printNumberOfSol(augmented_matrix):
    # get the matrix A by excluding vector b
    A = [i[:-1] for i in augmented_matrix]

    #using method numpy.linalg.matrix_rank() for calculating matrices' rank
    #to conclude the number of solutions, not used for finding the solutions
    rank_augmented = np.linalg.matrix_rank(augmented_matrix)
    rank_A = np.linalg.matrix_rank(A)

    num_variables = len(A[0]) #- 1  # Excluding the augmented column
    
    if rank_augmented == rank_A + 1:
        return ('No solutions.')
    elif rank_augmented == rank_A and rank_A == num_variables:
        return ('Unique solution.')
    else:
        return ('Infinite solutions.')
    

In [306]:
def main():
    print ("Enter the linear system equations (as known as an augmented matrix)\n")
    augmented_matrix = inputSystemOfEquations()
    print ("Your newly entered input is\n")
    for row in augmented_matrix:
        print (row)
    print ("\n")
    row_echelon_matrix = Gauss_elimination (augmented_matrix)

    print ("Echelon matrix after Gaussian elimination:\n")
    for row in row_echelon_matrix:
        print (row)
    print("\n The system of equations is in case of: ")
    
    numberOfSol = printNumberOfSol (augmented_matrix)
    if numberOfSol == 'No solutions.':
        print(numberOfSol)
        return
    else:
        print(numberOfSol)
        back_substitution(row_echelon_matrix)
            

## Test the program
\
Users enter each equation one by one, and then the program will print the entered augmented matrix, row echelon matrix after Gaussian elimination and notify the number of solutions with its expression or values. 

In [313]:
if __name__ == "__main__":
    main()

Enter the linear system equations (as known as an augmented matrix)



Enter the number of equations:  3
Enter the 1th equation (e.g.: '3 6 9' for 3x + 6y = 9):  2 -3 4 -1 0
Enter the 2th equation (e.g.: '3 6 9' for 3x + 6y = 9):  6 1 -8 9 0
Enter the 3th equation (e.g.: '3 6 9' for 3x + 6y = 9):  2 6 1 -1 0


Your newly entered input is

[2.0, -3.0, 4.0, -1.0, 0.0]
[6.0, 1.0, -8.0, 9.0, 0.0]
[2.0, 6.0, 1.0, -1.0, 0.0]


Echelon matrix after Gaussian elimination:

[2.0, -3.0, 4.0, -1.0, 0.0]
[0.0, 10.0, -20.0, 12.0, 0.0]
[0.0, 0.0, 15.0, -10.8, 0.0]

 The system of equations is in case of: 
Infinite solutions.
x_1 = -0.5799999999999998*t_1 + 0.0
x_2 = 0.2400000000000002*t_1 + 0.0
x_3 = 0.7200000000000001*t_1 + 0.0
x_4 = 1*t_1 + 0


## Extended task
Using the library `sympy` with methods `symbols` (to name the variables), `Eq` (to mark the equations), `solve` (to solve and print the final result).

In [325]:
import sympy as sp

# Define the variables
x, y, z,t = sp.symbols('x y z t')

# Define the equations
eq1 = sp.Eq(2*x - 3*y + 4*z - 1*t, 0)
eq2 = sp.Eq(6*x +1*y -8*z +9*t, 0)
eq3 = sp.Eq(2*x + 6*y + 1*z-1*t, 0)

# Solve the linear system of equations
solution = sp.solve((eq1, eq2,eq3), (x, y, z,t))
solution
solution_decimal = {key: value.evalf() for key, value in solution.items()}
solution_decimal

{x: -0.58*t, y: 0.24*t, z: 0.72*t}

## Quick test
You can test the program with the following augmented matrix (uncomment one matrix to run)

In [339]:
#Unique sol
"""
test_augmented_matrix = [
    [1,1,2,3,1],
    [2,3,-1,-1,-6],
    [3,-1,-1,-2,-4],
    [1,2,3,-1,-4]
]
"""

#Inf
test_augmented_matrix =[
    [1,-2,2,7,-3,1],
    [-6,-5,15,3,4,2],
    [-5,-2,4,-1,1,-1],
    [-20,14,8,-16,50,7]
]

# No sol
"""
test_augmented_matrix =[
    [2,-2,1,-1,1,1],
    [1,2,-1,1,-2,1],
    [4,-10,5,-5,7,1],
    [2,-14,7,-7,11,1]
]
"""
test_echelon_matrix = Gauss_elimination (test_augmented_matrix)
print('Echelon matrix:')
for i in test_echelon_matrix:
    print (i)

print('\n')
numberOfSol = printNumberOfSol (test_augmented_matrix)
if numberOfSol == 'No solutions.':
    print(numberOfSol)
else:
    print(numberOfSol)
    back_substitution(test_echelon_matrix)
            


Echelon matrix:
[1, -2, 2, 7, -3, 1]
[0.0, -17.0, 27.0, 45.0, -14.0, 8.0]
[0.0, 0.0, -5.0588235294117645, 2.235294117647058, -4.117647058823529, -1.6470588235294121]
[0.0, 0.0, 0.0, 58.13953488372094, 5.953488372093018, 12.581395348837209]


Infinite solutions.
x_1 = 0.5167999999999999*t_1 + 0.18520000000000025
x_2 = -2.4591999999999996*t_1 + 0.7712
x_3 = -0.8592*t_1 + 0.4212000000000001
x_4 = -0.1023999999999999*t_1 + 0.21639999999999995
x_5 = 1*t_1 + 0


In [333]:
# Check the with sympy: copy the augmented matrix and paste (results in the fraction form, they are equal)
x, y, z, t, u = sp.symbols('x y z t u')


augmented_matrix = sp.Matrix([
   [1,-2,2,7,-3,1],
    [-6,-5,15,3,4,2],
    [-5,-2,4,-1,1,-1],
    [-20,14,8,-16,50,7]
])


solution = sp.linsolve(augmented_matrix, x, y, z, t, u)
solution

{(323*u/625 + 463/2500, 482/625 - 1537*u/625, 1053/2500 - 537*u/625, 541/2500 - 64*u/625, u)}

## Descriptions for functions' implementations 
*Note*: Gauss elimination and back substitution are in the beginning as the algorithms) 
- `countAllZerosBeforeNonZero`: count all entries in a row are zero until meets a non-zero entry, to identify the first non-zero leading entry.
- `checkNotAllZero`: check if a row has all elements are zeros (used to check echelon matrix).
- `isEchelonMatrix`: check if a row echelon matrix, as a condition to stop and return in `Gauss_elimination`. 
- `swapRow`: move rows with less 0s to the top, to ensure the characteristic of row echelon matrix: the first non-zero of row[i+1] is to the right of row[i]'s.
- `format_sequence`: display the solutions in case of infinite solutions.
- `printNumberOfSol`: use `Kronecker-Capelli` Theorem and compare the rank of augmented matrix and A to conclude the number of the linear system equations.


## Special Note:
The descriptions and ideas for functions' implementations are also noted and explained in details before and in comment parts of each one for easier comprehension and traceability. 

**THANK YOU FOR USING MY FIRST PROGRAM WITH JUPYTER NOTEBOOK!**